## Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Filter the DataFrame using input statements for minimum and maximum temperature preferences.
2. Drop any empty rows on the new DataFrame. 
3. Find the nearest hotel to each city using each city’s latitude and longitude and the Google Nearby Search.  
4. Create a new DataFrame where the rows that do not have a hotel are dropped from the DataFrame created in Step 4. 
5. Add the cities to a marker layer map with a pop-up marker for each city that includes: 
    - Hotel name
    - City
    - Country
    - Current weather description and the maximum temperature
6. Export the DataFrame as a CSV file into the Vacation_Search folder as WeatherPy_vacation.csv.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,27.72,70,100,8.99,snow
1,1,Mogadishu,SO,2.0371,45.3438,79.23,76,32,21.00,scattered clouds
2,2,Hermanus,ZA,-34.4187,19.2345,83.80,66,11,13.60,few clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,69.01,55,100,18.99,moderate rain
4,4,Udachnyy,RU,66.4167,112.4000,-7.83,97,100,12.93,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria.. 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 65
What is the maximum temperature you would like for your vacation? 90


In [6]:
# 3. Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mogadishu,SO,2.0371,45.3438,79.23,76,32,21.00,scattered clouds
2,2,Hermanus,ZA,-34.4187,19.2345,83.80,66,11,13.60,few clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,69.01,55,100,18.99,moderate rain
11,11,Araouane,ML,18.9048,-3.5265,78.40,9,3,15.21,clear sky
12,12,Karratha,AU,-20.7377,116.8463,81.73,72,37,12.62,scattered clouds
17,17,Castro,BR,-24.7911,-50.0119,75.00,72,82,9.33,broken clouds
19,19,Kutum,SD,14.2000,24.6667,73.62,18,3,10.74,clear sky
21,21,Puerto Ayora,EC,-0.7393,-90.3518,75.16,79,80,6.13,broken clouds
22,22,Bengkulu,ID,-3.8004,102.2655,77.59,88,100,3.98,light rain
23,23,Pasni,PK,25.2631,63.4710,71.76,56,0,5.01,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                292
City                   292
Country                287
Lat                    292
Lng                    292
Max Temp               292
Humidity               292
Cloudiness             292
Wind Speed             292
Current Description    292
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mogadishu,SO,2.0371,45.3438,79.23,76,32,21.00,scattered clouds
2,2,Hermanus,ZA,-34.4187,19.2345,83.80,66,11,13.60,few clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,69.01,55,100,18.99,moderate rain
11,11,Araouane,ML,18.9048,-3.5265,78.40,9,3,15.21,clear sky
12,12,Karratha,AU,-20.7377,116.8463,81.73,72,37,12.62,scattered clouds


In [9]:
clean_travel_cities.count()

City_ID                287
City                   287
Country                287
Lat                    287
Lng                    287
Max Temp               287
Humidity               287
Cloudiness             287
Wind Speed             287
Current Description    287
dtype: int64

In [10]:
clean_travel_cities.columns

Index(['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Current Description'],
      dtype='object')

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,79.23,scattered clouds,2.0371,45.3438,
2,Hermanus,ZA,83.80,few clouds,-34.4187,19.2345,
3,Mar Del Plata,AR,69.01,moderate rain,-38.0023,-57.5575,
11,Araouane,ML,78.40,clear sky,18.9048,-3.5265,
12,Karratha,AU,81.73,scattered clouds,-20.7377,116.8463,
17,Castro,BR,75.00,broken clouds,-24.7911,-50.0119,
19,Kutum,SD,73.62,clear sky,14.2000,24.6667,
21,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
22,Bengkulu,ID,77.59,light rain,-3.8004,102.2655,
23,Pasni,PK,71.76,clear sky,25.2631,63.4710,


In [12]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,79.23,scattered clouds,2.0371,45.3438,Hotel Juba
2,Hermanus,ZA,83.80,few clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
3,Mar Del Plata,AR,69.01,moderate rain,-38.0023,-57.5575,Gran Hotel Mar del Plata
11,Araouane,ML,78.40,clear sky,18.9048,-3.5265,
12,Karratha,AU,81.73,scattered clouds,-20.7377,116.8463,ibis Styles Karratha
17,Castro,BR,75.00,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
19,Kutum,SD,73.62,clear sky,14.2000,24.6667,Dibbo areaمنطقة دبو
21,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
22,Bengkulu,ID,77.59,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
23,Pasni,PK,71.76,clear sky,25.2631,63.4710,Kashmiri Hotal


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,79.23,scattered clouds,2.0371,45.3438,Hotel Juba
2,Hermanus,ZA,83.80,few clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
3,Mar Del Plata,AR,69.01,moderate rain,-38.0023,-57.5575,Gran Hotel Mar del Plata
12,Karratha,AU,81.73,scattered clouds,-20.7377,116.8463,ibis Styles Karratha
17,Castro,BR,75.00,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
19,Kutum,SD,73.62,clear sky,14.2000,24.6667,Dibbo areaمنطقة دبو
21,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
22,Bengkulu,ID,77.59,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
23,Pasni,PK,71.76,clear sky,25.2631,63.4710,Kashmiri Hotal
24,Sawang Daen Din,TH,79.07,clear sky,17.4753,103.4575,บ้านเจเน็ต คอมพิวเตอร์


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

###  Add a pop-up markers to each city. 
---
#### In the pop-up for each city add: 
1. The city name
2. The country code
3. The weather description and maximum temperature for the city

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))